In [1]:
# 스파크 세션 생성
import pyspark
from pyspark.sql import SparkSession

# SynapseML : 마이크로소프트에서 만든 스파크용 ML 라이브러리
# https://microsoft.github.io/SynapseML/
# config에서 synapseml 모듈 불러오기
spark = SparkSession.builder \
    .appName("Spark study - 231228") \
    .config("spark.jars.packages", "com.microsoft.azure:synapseml_2.12:1.0.2") \
    .getOrCreate()

24/01/04 16:23:45 WARN Utils: Your hostname, KJH-DESKTOP resolves to a loopback address: 127.0.1.1; using 192.168.69.220 instead (on interface eth0)
24/01/04 16:23:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/kjh/app/spark/3.5.0/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/kjh/.ivy2/cache
The jars for the packages stored in: /home/kjh/.ivy2/jars
com.microsoft.azure#synapseml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7797c582-eca4-4218-9cc9-80635f7a0279;1.0
	confs: [default]
	found com.microsoft.azure#synapseml_2.12;1.0.2 in central
	found com.microsoft.azure#synapseml-core_2.12;1.0.2 in central
	found org.apache.spark#spark-avro_2.12;3.4.1 in central
	found org.tukaani#xz;1.9 in central
	found commons-lang#commons-lang;2.6 in central
	found org.scalactic#scalactic_2.12;3.2.14 in central
	found org.scala-lang#scala-reflect;2.12.15 in central
	found io.spray#spray-json_2.12;1.3.5 in central
	found com.jcraft#jsch;0.1.54 in central
	found org.apache.httpcomponents.client5#httpclient5;5.1.3 in central
	found org.apache.httpcomponents.core5#httpcore5;5.1.3 in central
	found org.apache.httpcomponents.core5#httpcore5-h2;5.1.3 in central
	found org.slf4j#slf4j-api;1.7.25 in centra

In [2]:
import requests, os

In [3]:
def get_access_token(client_id:str, client_sc:str):
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
    }
    data = f'grant_type=client_credentials&client_id={client_id}&client_secret={client_sc}'.encode()
    response = requests.post('https://accounts.spotify.com/api/token', headers=headers, data=data).json()
    access_token = response['access_token']

    return access_token

def get_response(access_token:str, endpoint:str, params:dict=None):
    import requests, json

    url = f"https://api.spotify.com/v1/{endpoint}"
    headers = {
        'Authorization': f'Bearer {access_token}',
    }

    if params != None:
        response = requests.get(url=url, params=params, headers=headers)
    else:
        response = requests.get(url=url, headers=headers)
    
    print(response)
    
    if response.status_code == 200:
        try:
            data = response.json()
            return data
        except json.decoder.JSONDecodeError:
            raise ValueError(f"API Server Error - {endpoint} - Invalid JSON content in response: {response.text}")
    else:
        raise ValueError(f"API Server Error - {endpoint} - Non-200 status code received: {response.status_code}")
    
def post_response(access_token:str, endpoint:str, data:dict=None):
    import requests, json

    url = f"https://api.spotify.com/v1/{endpoint}"
    headers = {
        'Authorization': f'Bearer {access_token}',
    }

    response = requests.post(url=url, headers=headers)
    
    print(response)
    
    if response.status_code == 200:
        pass
    else:
        raise ValueError(f"API Server Error - {endpoint} - Non-200 status code received: {response.status_code}")

In [4]:
from configparser import ConfigParser
current_dir = os.getcwd()
config_dir = os.path.join(current_dir, f'../config/config.ini')

parser = ConfigParser()
parser.read(config_dir)
access_key = parser.get("SPOTIFY", "ACCESS")
secret_key = parser.get("SPOTIFY", "SECRET")

In [5]:
access_token = get_access_token(access_key, secret_key)

In [6]:
user_id = "317sdvs4ldhgrv63cstoyzf6vo2y"

In [7]:
# get user's playlists
endpoint_pl = f"users/{user_id}/playlists"
params_pl = {
    "limit": 50,
    "offset": 0
}
playlists = get_response(access_token=access_token, endpoint=endpoint_pl, params=params_pl)

<Response [200]>


In [8]:
for item in playlists['items']:
    if item['name'] == '현재':
        playlist_id = item['id']
        break

In [9]:
# get playlist info
endpoint_plinfo = f"playlists/{playlist_id}"
playlist_info = get_response(access_token=access_token, endpoint=endpoint_plinfo)

<Response [200]>


In [10]:
playlist_info

{'collaborative': False,
 'description': '',
 'external_urls': {'spotify': 'https://open.spotify.com/playlist/0aQppRzTXAqSYxjL4Z4Kcm'},
 'followers': {'href': None, 'total': 0},
 'href': 'https://api.spotify.com/v1/playlists/0aQppRzTXAqSYxjL4Z4Kcm',
 'id': '0aQppRzTXAqSYxjL4Z4Kcm',
 'images': [{'height': 640,
   'url': 'https://mosaic.scdn.co/640/ab67616d0000b27328aa7897fae9255b2b3142b7ab67616d0000b27374fc13434c785c0b745bdba9ab67616d0000b2739192b5bd1205736587cd1bc3ab67616d0000b273cd1b9fe1c4476e81611cde16',
   'width': 640},
  {'height': 300,
   'url': 'https://mosaic.scdn.co/300/ab67616d0000b27328aa7897fae9255b2b3142b7ab67616d0000b27374fc13434c785c0b745bdba9ab67616d0000b2739192b5bd1205736587cd1bc3ab67616d0000b273cd1b9fe1c4476e81611cde16',
   'width': 300},
  {'height': 60,
   'url': 'https://mosaic.scdn.co/60/ab67616d0000b27328aa7897fae9255b2b3142b7ab67616d0000b27374fc13434c785c0b745bdba9ab67616d0000b2739192b5bd1205736587cd1bc3ab67616d0000b273cd1b9fe1c4476e81611cde16',
   'width': 60}]

In [12]:
from pyspark.sql import Row

# 'items' 데이터 추출
items_data = playlist_info['tracks']['items']

# Row 객체로 변환 (필요에 따라)
items_rows = [Row(**item) for item in items_data]

In [15]:
import json

# JSON 문자열로 변환
json_string  = json.dumps(playlist_info)

# JSON 문자열을 RDD로 변환
json_rdd = spark.sparkContext.parallelize([json_string])

# RDD를 사용하여 DataFrame 생성 (multiline 옵션 사용)
df_plinfo = spark.read.json(json_rdd, multiLine=True)

24/01/04 17:48:46 WARN GpuOverrides: 
! <SerializeFromObjectExec> cannot run on GPU because not all expressions can be replaced; GPU does not currently support the operator class org.apache.spark.sql.execution.SerializeFromObjectExec
  @Expression <Alias> staticinvoke(class org.apache.spark.unsafe.types.UTF8String, StringType, fromString, input[0, java.lang.String, true], true, false, true) AS value#2 could run on GPU
    ! <StaticInvoke> staticinvoke(class org.apache.spark.unsafe.types.UTF8String, StringType, fromString, input[0, java.lang.String, true], true, false, true) cannot run on GPU because GPU does not currently support the operator class org.apache.spark.sql.catalyst.expressions.objects.StaticInvoke
      ! <BoundReference> input[0, java.lang.String, true] cannot run on GPU because GPU does not currently support the operator class org.apache.spark.sql.catalyst.expressions.BoundReference
  ! <ExternalRDDScanExec> cannot run on GPU because not all expressions can be replaced; 

In [16]:
df_plinfo.show()

24/01/04 17:48:58 WARN GpuOverrides: 
!Exec <CollectLimitExec> cannot run on GPU because the Exec CollectLimitExec has been disabled, and is disabled by default because Collect Limit replacement can be slower on the GPU, if huge number of rows in a batch it could help by limiting the number of rows transferred from GPU to CPU. Set spark.rapids.sql.exec.CollectLimitExec to true if you wish to enable it
  @Partitioning <SinglePartition$> could run on GPU
    ! <RDDScanExec> cannot run on GPU because GPU does not currently support the operator class org.apache.spark.sql.execution.RDDScanExec
      @Expression <AttributeReference> collaborative#6 could run on GPU
      @Expression <AttributeReference> description#7 could run on GPU
      @Expression <AttributeReference> external_urls#8 could run on GPU
      @Expression <AttributeReference> followers#9 could run on GPU
      @Expression <AttributeReference> href#10 could run on GPU
      @Expression <AttributeReference> id#11 could run on 

+-------------+-----------+--------------------+---------+--------------------+--------------------+--------------------+----+----------------------+-------------+------+--------------------+--------------------+--------+--------------------+
|collaborative|description|       external_urls|followers|                href|                  id|              images|name|                 owner|primary_color|public|         snapshot_id|              tracks|    type|                 uri|
+-------------+-----------+--------------------+---------+--------------------+--------------------+--------------------+----+----------------------+-------------+------+--------------------+--------------------+--------+--------------------+
|        false|           |{https://open.spo...|{NULL, 0}|https://api.spoti...|0aQppRzTXAqSYxjL4...|[{640, https://mo...|현재|{주헌, {https://ope...|         NULL|  true|MTQ1LGJhY2UzMGRiZ...|{https://api.spot...|playlist|spotify:playlist:...|
+-------------+-----------+-----

In [17]:
df_plinfo.printSchema()

root
 |-- collaborative: boolean (nullable = true)
 |-- description: string (nullable = true)
 |-- external_urls: struct (nullable = true)
 |    |-- spotify: string (nullable = true)
 |-- followers: struct (nullable = true)
 |    |-- href: string (nullable = true)
 |    |-- total: long (nullable = true)
 |-- href: string (nullable = true)
 |-- id: string (nullable = true)
 |-- images: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- height: long (nullable = true)
 |    |    |-- url: string (nullable = true)
 |    |    |-- width: long (nullable = true)
 |-- name: string (nullable = true)
 |-- owner: struct (nullable = true)
 |    |-- display_name: string (nullable = true)
 |    |-- external_urls: struct (nullable = true)
 |    |    |-- spotify: string (nullable = true)
 |    |-- href: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- type: string (nullable = true)
 |    |-- uri: string (nullable = true)
 |-- primary_color: st

In [18]:
from pyspark.sql.functions import explode

df_plinfo = df_plinfo.select(explode('tracks.items').alias('items'))

In [19]:
df_plinfo.show()

24/01/04 17:55:32 WARN GpuOverrides: 
!Exec <CollectLimitExec> cannot run on GPU because the Exec CollectLimitExec has been disabled, and is disabled by default because Collect Limit replacement can be slower on the GPU, if huge number of rows in a batch it could help by limiting the number of rows transferred from GPU to CPU. Set spark.rapids.sql.exec.CollectLimitExec to true if you wish to enable it
  @Partitioning <SinglePartition$> could run on GPU
          ! <RDDScanExec> cannot run on GPU because GPU does not currently support the operator class org.apache.spark.sql.execution.RDDScanExec
            @Expression <AttributeReference> collaborative#6 could run on GPU
            @Expression <AttributeReference> description#7 could run on GPU
            @Expression <AttributeReference> external_urls#8 could run on GPU
            @Expression <AttributeReference> followers#9 could run on GPU
            @Expression <AttributeReference> href#10 could run on GPU
            @Expressio

+--------------------+
|               items|
+--------------------+
|{2023-04-16T22:07...|
|{2023-04-16T22:08...|
|{2023-04-17T10:34...|
|{2023-04-26T15:08...|
|{2023-04-26T15:18...|
|{2023-07-31T21:34...|
|{2023-07-31T21:35...|
|{2023-07-31T21:53...|
|{2023-08-06T15:22...|
|{2023-08-06T15:23...|
|{2023-09-21T11:27...|
|{2023-09-21T11:27...|
|{2023-09-24T05:31...|
|{2023-09-24T05:32...|
|{2023-09-24T05:32...|
|{2023-09-24T05:33...|
|{2023-09-24T05:33...|
|{2023-11-06T03:50...|
|{2023-11-06T03:51...|
|{2023-11-06T03:52...|
+--------------------+
only showing top 20 rows



In [20]:
df_plinfo = df_plinfo.select("items.*")

In [21]:
df_plinfo.show()

24/01/04 17:58:02 WARN GpuOverrides: 
!Exec <CollectLimitExec> cannot run on GPU because the Exec CollectLimitExec has been disabled, and is disabled by default because Collect Limit replacement can be slower on the GPU, if huge number of rows in a batch it could help by limiting the number of rows transferred from GPU to CPU. Set spark.rapids.sql.exec.CollectLimitExec to true if you wish to enable it
  @Partitioning <SinglePartition$> could run on GPU
          ! <RDDScanExec> cannot run on GPU because GPU does not currently support the operator class org.apache.spark.sql.execution.RDDScanExec
            @Expression <AttributeReference> collaborative#6 could run on GPU
            @Expression <AttributeReference> description#7 could run on GPU
            @Expression <AttributeReference> external_urls#8 could run on GPU
            @Expression <AttributeReference> followers#9 could run on GPU
            @Expression <AttributeReference> href#10 could run on GPU
            @Expressio

+--------------------+--------------------+--------+-------------+--------------------+---------------+
|            added_at|            added_by|is_local|primary_color|               track|video_thumbnail|
+--------------------+--------------------+--------+-------------+--------------------+---------------+
|2023-04-16T22:07:33Z|{{https://open.sp...|   false|         NULL|{{single, [{{http...|         {NULL}|
|2023-04-16T22:08:10Z|{{https://open.sp...|   false|         NULL|{{single, [{{http...|         {NULL}|
|2023-04-17T10:34:26Z|{{https://open.sp...|   false|         NULL|{{album, [{{https...|         {NULL}|
|2023-04-26T15:08:58Z|{{https://open.sp...|   false|         NULL|{{album, [{{https...|         {NULL}|
|2023-04-26T15:18:41Z|{{https://open.sp...|   false|         NULL|{{compilation, [{...|         {NULL}|
|2023-07-31T21:34:55Z|{{https://open.sp...|   false|         NULL|{{single, [{{http...|         {NULL}|
|2023-07-31T21:35:15Z|{{https://open.sp...|   false|         NUL